In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash
import numpy as np

import pandas as pd

app = JupyterDash(__name__)

df = pd.read_csv('../query_results/country_indicators.csv')

app.layout = html.Div([
    html.Div([
        
        html.Div([
            dcc.Dropdown(
                np.insert(df['Country Name'].unique(), 0, 'All Countries'),
                'All Countries',
                id='country-name',
                multi=True,
                searchable=True,
            ),
        ], style={'width': '30%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Fertility rate, total (births per woman)',
                id='xaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='xaxis-type',
                inline=True
            )
        ], style={'width': '30%', 'display': 'inline-block'}),
        

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Life expectancy at birth, total (years)',
                id='yaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='yaxis-type',
                inline=True
            )
        ], style={'width': '30%', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='indicator-graphic'),
    
    dcc.RangeSlider(
        min=df['Year'].min(), 
        max=df['Year'].max(), 
        step=5, 
        value=[df['Year'].min()+10, df['Year'].max()-10], 
        id='year-range-slider',
        marks={str(year): str(year) for year in df['Year'].unique()},
    )
])


@app.callback(
    Output('indicator-graphic', 'figure'),
    Input('country-name', 'value'),
    Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value'),
    Input('xaxis-type', 'value'),
    Input('yaxis-type', 'value'),
    Input('year-range-slider', 'value'))
def update_graph(country_name, xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type, year_value):
    
    print (country_name, xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type, year_value)
    
    
    if  'All Countries' in country_name :
        dff = df[(df['Year'] >= year_value[0]) & (df['Year'] <= year_value[1])]
        dff.set_index('Year', inplace=True, drop=False)
        print(dff.head(5))
        # dff = dff.sort_values(by='Year')
        print(dff.columns)


        fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
                         y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
                         hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name']+'_'\
                         +dff[dff['Indicator Name'] == yaxis_column_name]['Year'].apply(str),
                         #hover_data=dff[dff['Indicator Name'] == yaxis_column_name]['Year'],
                         color=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name']
                        )
        
        print('If Executed')

        fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

        fig.update_xaxes(title=xaxis_column_name,
                         type='linear' if xaxis_type == 'Linear' else 'log')

        fig.update_yaxes(title=yaxis_column_name,
                         type='linear' if yaxis_type == 'Linear' else 'log')
    else:
        dff = df[(df['Country Name'].isin(country_name)) &\
                 (df['Year'] >= year_value[0]) & (df['Year'] <= year_value[1])]
        
        dff.set_index('Year', inplace=True, drop=False)
        print(dff.columns)
        

        """fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
                         y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
                         hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Year'])"""
        
        fig = px.line(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],\
                y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],\
                text = dff[dff['Indicator Name'] == yaxis_column_name]['Year'],\
                color=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])
        
        print('Else Executed')

        fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

        fig.update_xaxes(title=xaxis_column_name,
                         type='linear' if xaxis_type == 'Linear' else 'log')

        fig.update_yaxes(title=yaxis_column_name,
                         type='linear' if yaxis_type == 'Linear' else 'log')
        

    return fig

In [ ]:
app.run(debug=False, port='8896')